In [ ]:
from math import tau

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import scipy.linalg as LA
import scipy.signal as ss

from utils import *

plt.style.use('seaborn-notebook')
np.random.seed(293710966)

# Generazione del Segnale
$$
x(n) = 300 \, cos \left(2 \pi n \frac{f_0}{f_s} + 0.1 \right) + w(n)
$$

In [ ]:
power_freq = 50
sampling_freq = 1130
time = np.arange(4096)
harmonic_numbers = np.array([1])
no_of_harmonics = harmonic_numbers.max()

noise = np.random.normal(0, 0.1, time.size)
omega = tau * power_freq / sampling_freq
signal = 300. * np.cos(omega * time) + noise

fig, ax = plt.subplots()
ax.plot(time[:200], signal[:200].real)
ax.set_xlabel('time [s]')
ax.set_ylabel('signal [V]')
fig.tight_layout()

## Finestre di Osservazione

In [ ]:
fastest_period = np.around(sampling_freq / power_freq).astype(int)
time_window = fastest_period * no_of_harmonics
data_size = signal.size - time_window + 1

windows = [signal[i : i + time_window] for i in range(data_size)]
data_matrix = np.vstack(windows)

print(f'numero di finestre: {data_matrix.shape[0]}')
print(f'larghezza finestre: {data_matrix.shape[1]}')

## Matrice di Autorcorrelazione
$$
\hat{R}_v = \frac{1}{N} \mathbb{V}^H \mathbb{V}
$$

In [ ]:
correlation = data_matrix.T @ data_matrix / data_matrix.shape[0]

## Principal Component Analysis

In [ ]:
# signal: K largest      -> M - K : M - 1
# noise: M - K smallest  -> 0 : M - K - 1

signal_pca = LA.eigh(correlation, subset_by_index=(time_window - no_of_harmonics, time_window - 1))
noise_pca = LA.eigh(correlation, subset_by_index=(0, time_window - no_of_harmonics - 1))
signal_eigvecs = signal_pca[1].T
noise_eigvecs = noise_pca[1].T

## MUSIC
$$
P_{music}(e^{j\omega}) = \frac{1}{\sum_{i=K+1}^M |e^H s_i|^2}
$$

In [ ]:
omegas = np.linspace(0, np.pi, 2000)
freqs = omegas * sampling_freq / tau

steering_matrix = np.exp(1j * np.outer(omegas, np.arange(time_window)))
pseudo_power = 1 / np.sum(np.abs(noise_eigvecs @ steering_matrix.conj().T) ** 2, axis=0)
    
fig, ax = plt.subplots()
plotdb(ax, freqs, pseudo_power)
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Pseudospectrum [dB]')
fig.tight_layout()

In [ ]:
peaks_idx, _ = ss.find_peaks(pseudo_power)

peaks = pd.DataFrame()
peaks['omega'] = omegas[peaks_idx]
peaks['freq'] = freqs[peaks_idx]
peaks['power'] = pseudo_power[peaks_idx]
peaks = peaks.sort_values('power').tail(1).sort_values('omega')

fig, ax1 = plt.subplots()

plotdb(ax1, freqs, pseudo_power)
plotdb(ax1, peaks.freq, peaks.power, 'o')
ax1.set_xlabel('Frequency [Hz]')
ax1.set_ylabel('Pseudospectrum')

fig.tight_layout()

In [ ]:
noise_power = noise_pca[0].mean()
noise_power

In [ ]:
est_omegas = peaks.omega.values
est_steering_matrix = np.exp(1j * np.outer(est_omegas, np.arange(time_window)))

b = signal_pca[0][:harmonic_numbers.size] - noise_power
A = np.abs(signal_eigvecs[:harmonic_numbers.size] @ est_steering_matrix.conj().T) ** 2
powers = LA.solve(A, b)
amplitudes = np.sqrt(2 * powers)

data = zip([power_freq], peaks.freq, [300], amplitudes)
results = pd.DataFrame(data, columns=['real freq', 'est freq', 'real amp', 'est amp'])
results